<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/GESTION_ESTADO_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import warnings
import statsmodels.api as sm
import datetime
from datetime import date
from datetime import timedelta
from scipy import stats
warnings.filterwarnings("ignore")

In [172]:
!pip install pandas openpyxl

In [173]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Cargamos el consolidado del informe de operación de las subestaciones

In [201]:
datos = pd.read_excel('/content/drive/MyDrive/INFORME DE OPERACION STN/RESUMEN INFORME DE OPERACION STN_PYTHON.xlsx')

In [202]:
datos.shape

(2931, 25)

## GESTION DE PLANES DE TRABAJO A ESTADO EJECUTADO

In [ ]:
#Quitar los los planes de trabajo duplicados

In [204]:
datos_sin_duplicados=datos.drop_duplicates(datos[['PLAN DE TRABAJO']])
datos_sin_duplicados.shape

(1242, 25)

In [205]:
plan=datos_sin_duplicados[["NUMERO DE PERMISO","SUBESTACIÓN","FECHA DE INICIO","PLAN DE TRABAJO","TIPO DE PLAN"]]

In [206]:
pt=plan[["PLAN DE TRABAJO"]]

In [23]:
#Guardamos los planes de trabajo no repetidos para la consulta del estado en SAP

In [207]:
pt.to_csv("/content/drive/MyDrive/INFORME DE OPERACION STN/consulta_SAP.csv",index=False)

In [ ]:
#Cargamos la consulta en realizada en SAP de los estados de los planes

In [208]:
sap = pd.read_excel('/content/drive/MyDrive/INFORME DE OPERACION STN/sap_python.XLSX')

In [ ]:
#Seleccionamos las 04 primeras columnas

In [209]:
ptsap=sap[['Planes Trab.', 'Estado', 'Núm.Consignación', 'Fecha fin revisión']]
ptsap.columns

Index(['Planes Trab.', 'Estado', 'Núm.Consignación', 'Fecha fin revisión'], dtype='object')

In [210]:
cabecera=['PLAN DE TRABAJO', 'Estado', 'Núm.Consignación', 'Fecha fin revisión']

In [ ]:
#Cambiamos el nombre del Planes Trab a PLAN DE TRABAJO de la consulta hecha en SAP

In [211]:
ptsap.columns=cabecera

In [226]:
#Hacemos un merge entre los planes no repetidos(plan) y la consulta en sap(ptsap)

In [227]:
merged_pt = pd.merge(plan,ptsap, on='PLAN DE TRABAJO', how='inner')

In [228]:
merged_pt.head(2)

,NUMERO DE PERMISO,SUBESTACIÓN,FECHA DE INICIO,PLAN DE TRABAJO,TIPO DE PLAN,Estado,Núm.Consignación,Fecha fin revisión
0,64,CARHUAQUERO (ISAP),2023-08-26,R0084687,NACIONAL,8,N84687,2023-08-27
1,54,CARHUAQUERO (ISAP),2023-08-16,R0101338,SIN CONSIGNACIÓN,8,S00000004773,2023-08-17


In [ ]:
#Creamos una columna con la fecha de hoy

In [229]:
hoy = date.today()
merged_pt["hoy"]=hoy
merged_pt['hoy'] = pd.to_datetime(merged_pt['hoy'])

In [230]:
merged_pt.head(2)

,NUMERO DE PERMISO,SUBESTACIÓN,FECHA DE INICIO,PLAN DE TRABAJO,TIPO DE PLAN,Estado,Núm.Consignación,Fecha fin revisión,hoy
0,64,CARHUAQUERO (ISAP),2023-08-26,R0084687,NACIONAL,8,N84687,2023-08-27,2023-10-03
1,54,CARHUAQUERO (ISAP),2023-08-16,R0101338,SIN CONSIGNACIÓN,8,S00000004773,2023-08-17,2023-10-03


In [ ]:
#Filtramos los sin consignación y local que deben ser cambiados a estado EJECUTADO.

In [231]:
filtro1=merged_pt[(merged_pt.Estado<8) & ((merged_pt["Fecha fin revisión"])<(merged_pt["hoy"]))]
filtro1.head(3)

,NUMERO DE PERMISO,SUBESTACIÓN,FECHA DE INICIO,PLAN DE TRABAJO,TIPO DE PLAN,Estado,Núm.Consignación,Fecha fin revisión,hoy
38,483,CHICLAYO OESTE,2023-10-02,R0102226,SIN CONSIGNACIÓN,2,S00000004804,2023-10-02,2023-10-03


In [233]:
filtro2=filtro1["PLAN DE TRABAJO"]

In [ ]:
#Guardamos en el drive los planes que deben ser pasados a EJECUTADO

In [234]:
filtro2.to_csv("/content/drive/MyDrive/INFORME DE OPERACION STN/pasarPTa8.csv",index=False)

##CALCULO DE INDICADORES

In [235]:
merged_pt

,NUMERO DE PERMISO,SUBESTACIÓN,FECHA DE INICIO,PLAN DE TRABAJO,TIPO DE PLAN,Estado,Núm.Consignación,Fecha fin revisión,hoy
0,64,CARHUAQUERO (ISAP),2023-08-26,R0084687,NACIONAL,8,N84687,2023-08-27,2023-10-03
1,54,CARHUAQUERO (ISAP),2023-08-16,R0101338,SIN CONSIGNACIÓN,8,S00000004773,2023-08-17,2023-10-03
2,53,CARHUAQUERO (ISAP),2023-08-15,R0101302,LOCAL,8,L00000007690,2023-08-15,2023-10-03
3,51,CARHUAQUERO (ISAP),2023-08-11,R0100950,LOCAL,8,L00000007653,2023-08-15,2023-10-03
4,48,CARHUAQUERO (ISAP),2023-08-08,R0101155,LOCAL,8,L00000007677,2023-08-08,2023-10-03
...,...,...,...,...,...,...,...,...,...
1216,7,ZORRITOS (REP),2023-01-22,R0092776,NACIONAL,8,N92776,2023-01-22,2023-10-03
1217,6,ZORRITOS (REP),2023-01-22,R0092775,NACIONAL,8,N92775,2023-01-22,2023-10-03
1218,5,ZORRITOS (REP),2023-01-22,R0094282,NACIONAL,8,N94282,2023-01-22,2023-10-03
1219,4,ZORRITOS (REP),2023-01-22,R0096013,NACIONAL,8,N96013,2023-01-22,2023-10-03
